In [1]:
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import mlp_builder as bare_model_builder

PWD = '../../..'
sys.path.append(PWD)

In [2]:
PROJECT_NAME = 'mlp'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 4096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22134, 24), (17707, 24), (4427, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                300       
                                                                 
 dense_1 (Dense)             (None, 18)                234       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 553
Trainable params: 553
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 7, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 61153
========== Model 1 ==========
{'n_hidden_layers': 1, 'units_mult': 16, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1345
========== Model 2 ==========
{'n_hidden_layers': 6, 'units_mult': 2, 'unit_decrease_factor': 2.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 25363
========== Model 3 ==========
{'n_hidden_layers': 1, 'units_mult': 16, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5, 'normalize_signal': False}
Number of parameters: 1537
========== Model 4 ==========
{'n_hidden_layers': 8, 'units_m

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME, n_epochs=N_EPOCHS,
                                           batch_size=BATCH_SIZE, n_top=TOP_N, es_min_delta=ES_MIN_DELTA,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 7, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 61153
Got score: 8.7921 (8.8556, 8.7287)
Got score: 8.1171 (8.1111, 8.1232)
Got score: 8.2539 (8.2556, 8.2522)
Got score: 8.5283 (8.5699, 8.4866)
Got score: 8.1198 (8.1268, 8.1128)


{'n_hidden_layers': 1, 'units_mult': 16, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1345
Got score: 8.8190 (8.8497, 8.7882)
Got score: 8.2208 (8.1707, 8.2709)
Got score: 8.4386 (8.4703, 8.4070)
Got score: 8.5064 (8.5048, 8.5080)
Got score: 8.2481 (8.2227, 8.2735)


{'n_hidden_layers': 6, 'units_mult': 2, 'unit_decrease_factor': 2.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 25363
Got score: 9.0553 (8.8430, 9.2677)
Got score: 8.1560 (8.1147, 8.1973)
Got score: 8.3955 (8.4404, 8.3506)
Got score: 8.4882 (8.5453, 8.4311)
Got score: 8.1181 (8.0986, 8.1375)


{'n_hidden_layers': 1, 'units_mult': 16, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5, 'normalize_signal': False}
Number of parameters: 1537
Got score: 8.9581 (8.9262, 8.9899)
Got score: 8.1779 (8.2476, 8.1081)
Got score: 8.3520 (8.3737, 8.3303)
Got score: 8.4673 (8.4135, 8.5210)
Got score: 8.2350 (8.2246, 8.2453)


{'n_hidden_layers': 8, 'units_mult': 8, 'unit_decrease_factor': 1.5, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 204910
Got score: 9.0635 (9.0257, 9.1013)
Got score: 8.3266 (8.2990, 8.3542)
Got score: 14.1523 (19.7381, 8.5665)
Got score: 8.6452 (8.6186, 8.6717)
Got score: 8.4831 (8.3986, 8.5676)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.36,0.26,61153
1,8.45,0.22,1345
2,8.44,0.34,25363
3,8.44,0.28,1537
4,9.73,2.22,204910
